In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

In [3]:
!git clone https://github.com/Anil-Banjade/Nepali_Multi_Modal.git


Cloning into 'Nepali_Multi_Modal'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 613 (delta 13), reused 17 (delta 10), pack-reused 574 (from 1)
Receiving objects: 100% (613/613), 3.88 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (372/372), done.


In [4]:
!git fetch origin
!git reset --hard origin/main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [5]:
%cd Nepali_Multi_Modal

/content/Nepali_Multi_Modal


In [6]:
%%capture
!pip install -r requirements.txt

In [7]:
from src.multimodal_text_generation.main import main

Using device: cuda


In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import dataset,DataLoader

from src.multimodal_text_generation.config import config
from src.multimodal_text_generation.models.transformer import Transformer
from src.multimodal_text_generation.data.dataset import CaptionEmbeddingDataset, collate_fn
from src.multimodal_text_generation.utils.inference import run_inference
from src.multimodal_text_generation.trainer import train_model

In [9]:
def main(resume_checkpoint=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tokenizer = AutoTokenizer.from_pretrained('NepBERTa/NepBERTa')

    loaded_results = torch.load('/content/drive/MyDrive/Minor_project/aligned_caption_embeddings.pt')

    dataset = CaptionEmbeddingDataset(loaded_results,tokenizer)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

    model = Transformer(tokenizer)
    if resume_checkpoint:
      model.load_state_dict(torch.load(resume_checkpoint))

    num_epochs = 10
    train_model(model, dataloader, num_epochs, device)
    return model

In [10]:
if __name__ == "__main__":
    model=main('/content/drive/MyDrive/Minor_project/autoregressive_1024_v2.pt')
    # model=main()
    torch.save(model.state_dict(), '/content/drive/MyDrive/Minor_project/autoregressive_1024_v3.pt')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

<ipython-input-9-650fe51614b2>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_results = torch.load('/content/drive/MyDrive/Minor_project/aligned_caption_embeddings.

Epoch [0/10], Batch [0/10111], Loss: 0.4872
Epoch [0/10], Batch [1000/10111], Loss: 0.5651
Epoch [0/10], Batch [2000/10111], Loss: 0.8183
Epoch [0/10], Batch [3000/10111], Loss: 0.5984
Epoch [0/10], Batch [4000/10111], Loss: 0.6375
Epoch [0/10], Batch [5000/10111], Loss: 0.4301
Epoch [0/10], Batch [6000/10111], Loss: 0.4234
Epoch [0/10], Batch [7000/10111], Loss: 0.6645
Epoch [0/10], Batch [8000/10111], Loss: 0.6499
Epoch [0/10], Batch [9000/10111], Loss: 0.6897
Epoch [0/10], Batch [10000/10111], Loss: 0.8574
Epoch [0/10], Average Loss: 0.6521
Epoch [1/10], Batch [0/10111], Loss: 0.7238
Epoch [1/10], Batch [1000/10111], Loss: 0.6472
Epoch [1/10], Batch [2000/10111], Loss: 0.8369
Epoch [1/10], Batch [3000/10111], Loss: 0.8283
Epoch [1/10], Batch [4000/10111], Loss: 0.6845
Epoch [1/10], Batch [5000/10111], Loss: 0.3720
Epoch [1/10], Batch [6000/10111], Loss: 0.6705
Epoch [1/10], Batch [7000/10111], Loss: 0.9399
Epoch [1/10], Batch [8000/10111], Loss: 0.5901
Epoch [1/10], Batch [9000/1011

In [11]:
# if __name__ == "__main__":
#     # model=main()
#     torch.save(model.state_dict(), '/content/drive/MyDrive/Minor_project/autoregressive_gen_model.pt')


In [16]:
import numpy as np

In [21]:
def test():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    loaded_results = torch.load(
        '/content/drive/MyDrive/Minor_project/aligned_caption_embeddings.pt',
        weights_only=False
    )

    tokenizer = AutoTokenizer.from_pretrained('NepBERTa/NepBERTa')
    dataset = CaptionEmbeddingDataset(loaded_results,tokenizer)
    for i in range(0,500,5):
      test_caption, test_embedding = dataset[i]


      if isinstance(test_embedding, np.ndarray):
          test_embedding = torch.from_numpy(test_embedding)

      fused_embedding = test_embedding.clone().detach().to(device)

      model_path = '/content/drive/MyDrive/Minor_project/autoregressive_1024_v3.pt'
      generated_caption = run_inference(model_path, fused_embedding, device)

      print("Original:", test_caption)
      print("Generated:", generated_caption)

In [22]:
test()

Loading model...

Attempt 1/3
Original: गुलाबी लुगा लगाएको बच्चा प्रवेश गर्ने बाटोमा सिँढीको सेट माथि चढिरहेको छ।
Generated: रातो शरट लगाएको कटा र पहलो हागामा हिडछ ।
Loading model...

Attempt 1/3
Original: कालो कुकुर र दाग भएको कुकुर लडिरहेका छन्
Generated: एक वयकति आफनो साइकल लिएर पहाडमा हाम फालद
Loading model...

Attempt 1/3
Original: रंगले ढाकेकी एउटी सानी केटी कचौरामा हात लिएर रंगिएको इन्द्रेणीको अगाडि बसिरहेकी छिन्।
Generated: एक जना मानिसल आफनो टाउकोमा अभियान चिनह उठाउद ।
Loading model...

Attempt 1/3
Original: एउटा मानिस बेन्चमा सुतेको बेला उसको कुकुर उसको छेउमा बस्छ।
Generated: एक वयकति आफनो साइकल समातर हावामा माथि उफरनछ ।
Loading model...

Attempt 1/3
Original: सुन्तला रंगको टोपी लगाएको मान्छे केहि देखिरहेको छ।
Generated: एउटा खरो ककर घासमा दौडिरहको छ ।
Loading model...

Attempt 1/3
Original: एउटा बच्चा डोरीको जालमा खेल्दै।
Generated: एक वयकति सरफबोरडमा उभिएको छ र ठलो छालमा सवार छ ।
Loading model...

Attempt 1/3
Original: एउटा कालो र सेतो कुकुर सेतो बारले घेरिएको घाँसेको बगैंच